<a href="https://colab.research.google.com/github/danielrodriguez007/Data_Analysis/blob/main/WhatsApp_Chat_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 5.3 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=3434a90246247e3d5bf8847c2c2e9b701244e126cebd0ba617d3bd585a5751c6
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [4]:
import regex 
import pandas as pd
import numpy as np 
import emoji 
from collections import Counter 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [5]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = regex.match(pattern, s)
    if result:
        return True
    return False

def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [6]:
data = [] 
conversation = '/content/drive/MyDrive/Colab Notebooks/WhatsApp Chat with Mamor.txt'
with open(conversation, encoding="utf-8") as fp:
  fp.readline()
  messageBuffer = []
  date, time, author = None, None, None 
  while True:
    line = fp.readline() 
    if not line:
      break
    line = line.strip()
    if date_time(line):
      if len(messageBuffer) > 0:
        data.append([date, time, author, ' '.join(messageBuffer)])
      messageBuffer.clear()
      date, time, author, message = getDatapoint(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)      

In [7]:
df = pd.DataFrame(data, columns=["Date", "Time", "Author", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
print(df.tail(20))
print(df.info())
print(df.Author.unique())

            Date   Time            Author  \
32692 2022-08-09  16:44  Daniel Rodríguez   
32693 2022-08-09  16:50             Mamor   
32694 2022-08-09  16:53  Daniel Rodríguez   
32695 2022-08-09  16:54  Daniel Rodríguez   
32696 2022-08-09  16:54             Mamor   
32697 2022-08-09  16:54             Mamor   
32698 2022-08-09  16:55  Daniel Rodríguez   
32699 2022-08-09  16:55             Mamor   
32700 2022-08-09  16:56             Mamor   
32701 2022-08-09  16:56             Mamor   
32702 2022-08-09  16:56  Daniel Rodríguez   
32703 2022-08-09  16:57  Daniel Rodríguez   
32704 2022-08-09  16:57  Daniel Rodríguez   
32705 2022-08-09  16:57             Mamor   
32706 2022-08-09  16:57             Mamor   
32707 2022-08-09  16:57             Mamor   
32708 2022-08-09  16:58  Daniel Rodríguez   
32709 2022-08-09  18:21             Mamor   
32710 2022-08-09  18:26  Daniel Rodríguez   
32711 2022-08-09  18:32             Mamor   

                                                 Messa

In [8]:
total_messages = df.shape[0]
print(total_messages)

32712


In [9]:
media_messages = df[df["Message"]=='<Media omitted>'].shape[0] 
print(media_messages)

4187


In [17]:
def split_count(text):
  emojis_list = []
  data = regex.findall(r'\X', text)
  for word in data: 
    if any(char in emoji.UNICODE_EMOJI_ALIAS_ENGLISH for char in word):
       emoji_list.append(word)
    return emojis_list  
df['emoji'] = df["Message"].apply(split_count) 

emojis = sum(df['emoji'].str.len())
print(emojis)

AttributeError: ignored